## Spectral Clustering and Semi-Supervised Learning

#### This notebook contains different values and imports that can be used in this practical session.
#### Please keep the same variable names when provided in your report to make the work of teaching assistants easier. 
#### You can still change the values given or the sizes of the datasets treated if you believe it is usefull to illustrate your point.

#### You can directly edit the markdown boxes to add your comments and answers to the questions

In [ ]:
# Question 1 (imports and advised values):
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np

matplotlib.style.use('seaborn-notebook')

n_samples = 200 # You can change these values
noise_level_list = [.05, .1, .2] # You can change these values

### Question 1 : Complete the code in the box below

In [ ]:
#A dictionnary which maps the noise level to the correponding moon dataset (if it has already been generated)
noisy_moons = {} 
for lev in noise_level_list:
    noisy_moons[lev] = make_moons(n_samples, shuffle = False, noise=lev)
    
f, a = plt.subplots(1, len(noise_level_list))
f.subplots_adjust(wspace = 0.5, hspace=0.5)

for i, lev in enumerate(noisy_moons):
    a[i].scatter(noisy_moons[lev][0][:,0], noisy_moons[lev][0][:,1], c = noisy_moons[lev][1])
    a[i].set_title('Noise = %.2f' % lev)

In [ ]:
# Question 2 :
from sklearn.neighbors import kneighbors_graph
n_neighbors_list = [1, 5, 10] # You can change these values

### Question 2 : Complete the code in the box below

In [ ]:
def plot_kneighbors_graph(axes, graph, pts, noise, k):
    axes.scatter(pts[:, 0], pts[:, 1], marker='.')
    xidx, yidx = graph.nonzero()
    for i, j in zip(xidx, yidx):
        if i < j:
            axes.plot([pts[i, 0], pts[j, 0]], [pts[i, 1], pts[j, 1]], linewidth=0.5, c='g')
    axes.set_title('Noise =  %.2f, k = %.1f' % (noise, k),fontsize = 10)

In [ ]:
#Generate the adjacency matrices
kNN_graphs = {}
for k in n_neighbors_list:
    for lev in noise_level_list:
        kNN_graphs[k, lev] = kneighbors_graph(noisy_moons[lev][0], k, mode='distance', p=2)

f, a = plt.subplots(len(n_neighbors_list), len(noise_level_list))
f.subplots_adjust(wspace = 0.5, hspace=0.5)
for i, k in enumerate(n_neighbors_list):
    for j, lev in enumerate(noise_level_list):
        plot_kneighbors_graph(a[i, j], kNN_graphs[k, lev], noisy_moons[lev][0], lev, k)
        #a[i, j].imshow(kNN_graphs[k, lev].todense(), cmap = 'Greys', interpolation="none")
        #a[i, j].set_title('Noise =  %.2f, k = %.1f' % (lev, k),fontsize = 10)

### Question 3 : Optimization problem 

- Given a graph $\mathcal{G} = (E,V)$, let $D$ be its degree matrix and $W$ its weighted ajacency matrix, such that $L := D - W$ is the Laplacian of the graph. In the case when we consider a partition in 2 subsets $A,\bar{A}$, the Normalized Cut problem translates as the following optimization problem:



- $\min_A \{ f^\top L f \} \text{ s.t. } f_i = 
\begin{cases}
    \sqrt{\frac{\text{Vol}(\bar{A})}{\text{Vol}(A)}},& \text{if } v_i\in A\\
    \sqrt{\frac{\text{Vol}(A)}{\text{Vol}(\bar{A})}},& \text{if } v_i\in \bar{A}
\end{cases}, Df \bot \mathbb{1}, f^\top D f = \text{Vol}(V)$


- This problem is then relaxed as $\min_{f\in \mathbb{R}^n} f^\top L f \text{ s.t. } Df \bot \mathbb{1}, f^\top D f = \text{Vol}(V)$


### Question 4 : Complete the code in the box below



In [ ]:
from sklearn.cluster import SpectralClustering

f, a = plt.subplots(len(n_neighbors_list), len(noise_level_list))
f.subplots_adjust(wspace = 0.5, hspace=0.5)
labels = {}
for i, k in enumerate(n_neighbors_list):
    for j, l in enumerate(noise_level_list):
        spectral = SpectralClustering(n_clusters=2, eigen_solver='arpack', affinity="precomputed")
        labels[k, l] = spectral.fit_predict(kneighbors_graph(noisy_moons[l][0], k, mode='distance', p=2).todense())
        a[i, j].scatter(noisy_moons[l][0][:,0], noisy_moons[l][0][:,1], c = labels[k, l])
        a[i, j].set_title('Noise =  %.2f, k = %.1f' % (l, k), fontsize = 10)


It seems that increasing the number improves the robustness of the clustering.
However, since it makes the adjacency matrix denser, it also makes the complexity greater.

### Question 5 : Complete the code in the box below

In [ ]:
#The Frobenius inner product for matrices
def frobenius(A, B):
    return np.sum(np.multiply(A,B))

#Compute the matrix representation C of a given clustering
#ie Cij = 1 iff xi and xj are clustered together and i != j
def matrix_clustering(labels):
    n = len(labels)
    C = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            if (labels[i] == labels[j] and i != j):
                C[i,j] = 1
    return C

def cosine_similarity(C1, C2):
    return frobenius(C1, C2) / (np.linalg.norm(C1, 'fro')*np.linalg.norm(C2, 'fro'))

#Return the average cosine similarity between a reference clustering and bootstrapped clusterings over B runs
def cluster_stability(X, algo, clusters=2, B=10, f=0.8):
    n = X.shape[0]
    ref = algo(X, clusters)
    
    avg_similarity = 0
    
    for i in range(B):
        indices = np.random.choice(n, f * n, replace=False)
        bootstrapped = X[indices, :]
        c = algo(bootstrapped, clusters)
        avg_similarity += cosine_similarity(matrix_clustering(c), matrix_clustering(ref[indices]))
        
    return avg_similarity / B

### Question 6 : Complete the code in the box below

Let us plot the stability as a function of the number of neighbors used for spectral clustering, on noisy and non-noisy versions of the moon dataset.

In [ ]:
noise_free_stability = []
noisy_stability = []
n_neighbors_list = np.arange(1,11,2)
for k in n_neighbors_list:
    noisy_stability.append(
        cluster_stability(noisy_moons[.2][0], 
            (lambda X, clusters: SpectralClustering(n_clusters=clusters, 
                            affinity="precomputed").fit_predict(kneighbors_graph(X, k, mode='distance', p=2).todense())),
            B=20, f=0.8))
    noise_free_stability.append(
        cluster_stability(noisy_moons[.05][0], 
            (lambda X, clusters: SpectralClustering(n_clusters=clusters, 
                            affinity="precomputed").fit_predict(kneighbors_graph(X, k, mode='distance', p=2).todense())), 
            B=20, f=0.8))

plt.plot(n_neighbors_list, noisy_stability, label = "Noisy")
plt.plot(n_neighbors_list, noise_free_stability, label = "Noise-free")
plt.legend()

### Question 7 : Complete the code in the box below

In [ ]:
from sklearn.cluster import AgglomerativeClustering



### Experiment on MNIST

In [ ]:
# Add your code here

## Semi-Supervised Learning

Choice of the dataset used : **Precise** which dataset you chose and why it is relevant for the semi-supervised learning Task

Advised datasets :

*Breast Cancer Wisconsin (Diagnostic) Database*

*MNIST binary even vs odd (multiple clusters inside each class)*

Feel free to use other datasets if they are relevant

In [ ]:
from sklearn.datasets import load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split


### For all the next questions, use Cancer and Mnist classes to handle your data if you choose to use these one,
### You can also add more datasets but we advise you to handle them with this class for better readability
class semi_sup_dat:
    def __init__(self,data,p_unlabelled,name):
        # DON T CHANGE THE RANDOM STATES
        self.name = name
        if self.name == 'Mnist':
            # do an even vs odd binary classification :
            even = [0,2,4,6,8]
            Y = [int(y in even) for y in data.target]
        else:
            Y = data.target
        X_lab, X_unlab, y_lab, y_unlab = train_test_split(data.data, data.target, test_size=p_unlabelled, random_state=32)
        self.X_lab = X_lab
        self.X_unlab = X_unlab
        self.y_lab = y_lab
        self.y_unlab = y_unlab


# The following lines can be called later in the code to build a dataset with varying unlabelled proportion
p_unlabelled = 0.8 # You can change this value
Cancer = semi_sup_dat(load_breast_cancer(), p_unlabelled,'Cancer')
digits = load_digits()

Mnist = semi_sup_dat(load_digits(), p_unlabelled,'Mnist')

### Question 9 : Complete the code in the box below

In [ ]:
#  Question 9  : Complete the function self_training
from sklearn.neighbors import KNeighborsClassifier





### Question 10 : Complete the code in the box below

###### Closed form expression :

-

-

-

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel # Or reimplement it yourself if your prefer
from scipy.linalg import block_diag

    

### Question 11 : Complete the code in the box below

### Question 12 : Add your answer here

-

-

-

### Question 13 : Complete the code in the box below

### Question 14 : Complete the code in the box below

##### Add your answer to the question here :

-

-

-

### Question 15 : Complete the code in the box below

### Question 16 : Complete the code in the box below

###### Describe your protocol here : 
-

-

-